# Using snowflake-connector-python

In [19]:
import snowflake.connector
import os
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

# Create a Snowflake connection object
try:
    conn = snowflake.connector.connect(user=os.getenv('SNOWFLAKE_USER'),
                                       password=os.getenv('SNOWFLAKE_PASSWORD'),   
                                       account=os.getenv('SNOWFLAKE_ACCOUNT'),
                                        database=os.getenv('SNOWFLAKE_DATABASE'),
                                        warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
                                        schema=os.getenv('SNOWFLAKE_SCHEMA'),
                                        table=os.getenv('SNOWFLAKE_TABLE')
    )
    print("Connection successfull.")
  
except Exception as e:
    print("Connection failed. Reason:", e)

Connection successfull.


In [20]:

print(os.getenv('SNOWFLAKE_TABLE'))

EMPLOYEE_JOB_FUNCTIONS


# Using SqlAlchemy

In [23]:
from sqlalchemy import create_engine
import os

from dotenv import load_dotenv

load_dotenv()
# see all the deprecation warnings
os.environ["SQLALCHEMY_WARN_20"] = "1"


user=os.getenv('SNOWFLAKE_USER')
password=os.getenv('SNOWFLAKE_PASSWORD') 
account=os.getenv('SNOWFLAKE_ACCOUNT')
database=os.getenv('SNOWFLAKE_DATABASE')
warehouse=os.getenv('SNOWFLAKE_WAREHOUSE')
schema=os.getenv('SNOWFLAKE_SCHEMA')
table=os.getenv('SNOWFLAKE_TABLE')
role=os.getenv('SNOWFLAKE_ROLE')

engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/{database}/{schema_name}/{warehouse}/{role}'.format(
        user=user,
        password=password,
        account_identifier=account,
    )
)
#'snowflake://<user_login_name>:<password>@<account_identifier>/<database_name>/<schema_name>?warehouse=<warehouse_name>&role=<role_name>'
try:
    connection = engine.connect()
    results = connection.execute('select * from DEMO_DB.STREAMABLE.EMPLOYEE_JOB_FUNCTIONS').fetchone()
    print(results[0])
finally:
    connection.close()
    engine.dispose()

KeyError: 'database'

In [14]:
# Using a connection string to simplify the connection process

engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/'.format(
        user=user,
        password=password,
        account_identifier=account,
    )
)
try:
    connection = engine.connect()
    results = connection.execute('select * from DEMO_DB.STREAMABLE.EMPLOYEE_JOB_FUNCTIONS').fetchone()
    print(results[0])
finally:
    connection.close()
    engine.dispose()


ProgrammingError: (snowflake.connector.errors.ProgrammingError) 002003 (02000): SQL compilation error:
Schema 'DEMO_DB.STREAMABLE' does not exist or not authorized.
[SQL: select * from DEMO_DB.STREAMABLE.EMPLOYEE_JOB_FUNCTIONS]
(Background on this error at: https://sqlalche.me/e/14/f405)

ResourceClosedError: This Connection is closed

In [4]:
# Add the table to the database
data_enterprise = [
        ['john.doe@musicprod.com', 1234567, 'Title & Curative'],        
        ['jane.smith@musicprod.com', 2345678, 'Policy'],
        ['david.wong@musicprod.com', 3456789, 'Recording'],
        ['sarah.jones@musicprod.com', 4567890, 'Document Intake'],
        ['alex.garcia@musicprod.com', 5678901, 'Closing']
    ]

    # create a pandas DataFrame with the specified column names
df = pd.DataFrame(data_enterprise, columns=['employee_email', 'employee_id', 'job_function_name']) 
    # convert all columns to str
df.employee_id = df.employee_id.astype(str) 
df.employee_email = df.employee_email.astype(str)  
df.job_function_name = df.job_function_name.astype("category")
df.job_function_name = df.job_function_name.cat.add_categories(("Compliance", "Scheduling", "Concierge", "Funding", "Post Closing"))
print(df)

              employee_email employee_id job_function_name
0     john.doe@musicprod.com     1234567  Title & Curative
1   jane.smith@musicprod.com     2345678            Policy
2   david.wong@musicprod.com     3456789         Recording
3  sarah.jones@musicprod.com     4567890   Document Intake
4  alex.garcia@musicprod.com     5678901           Closing


In [58]:
from snowflake.connector.pandas_tools import pd_writer
import pandas as pd
from sqlalchemy import create_engine




#Write the data to Snowflake, using pd_writer to speed up loading

with engine.connect() as con:
        df.to_sql(name=table_name.lower(), con=con, if_exists=if_exists, method=pd_writer)

ImportError: cannot import name 'string_types' from 'sqlalchemy.util' (/Users/jozer/Documents/GitHub/streamable-1/env_streamable-1/lib/python3.10/site-packages/sqlalchemy/util/__init__.py)

# Using sql-alchemy